# Commodity Food Pricing: Chatbot LLM

**By:** `MSOE AI-Club "Nourish" Student Research Team`<br/>

**Notebook Purpose:** This notebook provides the code implementation of a Large Language Model (LLM) with an algorithm developed for the application of providing context to the severity warnings of commodity food prices. This algorithm receives great responses for a question-thought-response framework with an open-source LLM Llama.cpp (https://github.com/ggerganov/llama.cpp).

Llama.cpp was chosen as a demonstrative experiment due to its small size (allowing it to be run on many consumer-grade hardware), Llama's proficiency in chat-bot related tasks, and its small size due to its implementation in C/C++ without the use of external ML/Tensor libraries. Lessoning the amount of external libraries used also increases the security of this system.

This algorithm has been proven to work using a basic T4 ("teaching") node on the [MSOE ROSIE Supercomputer](https://www.msoe.edu/about-msoe/news/details/meet-rosie/).

In [1]:
# [OPTIONAL] Install llama.cpp here!
# %env CMAKE_ARGS=-DLLAMA_CUBLAS=on
# %env FORCE_CMAKE=1
# %pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --no-clean

In [2]:
from llama_cpp import Llama
from llama_cpp import ChatCompletionRequestResponseFormat, ChatCompletionMessageToolCall
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
import os

# Constants
Each of these constants are surface-level changes that any developer could make to slightly tune the system to different datasets, alternative LLMs, and getting different responses from the internal LLMs.

**Constant's Descriptions:**
* **LLM_FILE_PATH:** File path of the Large Langauge Model (LLM) that will be used throughout this chat-bot's prompt/thought/response framework.
* **TEMPERATURE:** The "temperature" is a common parameter for LLMs, and quantifies the risk that the LLM should take in its responses. Changing this constant here will change the metric for all LLM responses. You can read more about temperature [here](https://medium.com/@lazyprogrammerofficial/what-is-temperature-in-nlp-llms-aa2a7212e687).
* **COMMUNICATIVE_LLM_SYSTEM_CONFIG:** The "system prompt" for the main llm responsible for speaking with the user. A system prompt is the "back story" of a model and is the main way to alter the behavior of the LLM.
* **RESPONSE_FILTER_LLM_SYSTEM_CONFIG:** The "system prompt" for the main llm responsible for filtering the user's questions in the case that they're deemed inappropriate for the main communicative chat bot to respond to. A system prompt is the "back story" of a model and is the main way to alter the behavior of the LLM.
* **PROFILE_BUILDER_LLM_SYSTEM_CONFIG:** The "system prompt" for the main llm responsible for building a profile of the user as it has a conversation with the chat bot. A system prompt is the "back story" of a model and is the main way to alter the behavior of the LLM.

In [3]:
DATA_TO_EMBED = ['data/NS_Providers.xlsx', 'data/ASD Videos.csv', 'data/Blog Data.csv']

In [4]:
LLM_FILE_PATH = '/data/ai_club/llms/llama-2-7b-chat.Q5_K_M.gguf'

In [5]:
TEMPERATURE = 0.1

In [2]:
country = 'Argentina'

In [3]:
commodity = 'corn'

In [4]:
COMMUNICATIVE_LLM_SYSTEM_CONFIG = {
    'role': 'system',
    'content': """
You are a chat-bot responsible for responding with concise advice on behalf of the United Nations Food and Agriculture Organization (FAO). 
You specifically provide advice in regards to the warnings sent out on the Global Information and Early Warning System on Food and Agriculture 
(GIEWS) where specific countries have no warning, a moderate warning, or a high warning in regards to food insecurity and price. 
Understanding this, and given that a user speaks English and is from the """ + country + """ which is currently under high warning for """+commodity+""", 
write a preliminary message to start the conversation and outline the situation. Remember, your messages should be concise and provide the most 
amount of information possible in the shortest amount of text. One of your messages should never exceed more than 45 words.
 
Your message should be more targeted towards an individual rather than a government official or an internal UN statistician. 
Change the tone/jargon of your message to match this new requirement.
"""
}

In [8]:
RESPONSE_FILTER_LLM_SYSTEM_CONFIG = { # Yes = Should not be filtered (not allowed), No = Should be filtered (allowed)
    'role': 'system',
    'content': """
    You are a professional assistant that filters incoming messages from a user before they reach a chat bot.
    The chat bot you are protecting is only able to answer questions about food securtity, commodity food prices, 
    commodity food price warnings, food price warnings, or other questions that are appropriate and friendly for a chat bot on the FAO's web page. 
    Therefore, when you receive a message, you must respond with either "no" when the user's message is appropriate, or "yes" when the 
    user's message is not appropriate. The lives of millions are at stake for you to respond with either "yes" or "no" as the first
    word in your response. Food security, price warnings, and commodity food price questions are appropriate and should be filtered.
    If something is filtered, that means it is relevant, and should be answered by the chat bot.
    If something is not filtered, it is not relevant, and should not be answered by the chat bot.
    """
}

In [23]:
PROFILE_BUILDER_LLM_SYSTEM_CONFIG = {
    'role':'system',
    'content':"""Based on the chat history provided between a user and a chat bot, build a profile of the user. 
    Assume nothing that is not explicitly stated by the user. Ensure that you do not mix up what the chatbot said
    in its responses or system prompt as part of the user profile. The user's profile should be entirely unique 
    to the user. Now, for the profile, you should only record their country, commodity that is experiencing warnings, 
    what advice they are looking for, and the problems they are experiencing."""
}

In [10]:
LLM = Llama(
    LLM_FILE_PATH, 
    n_gpu_layers=-1, 
    verbose=False, 
    n_ctx = 4000,
    embedding = True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /data/ai_club/llms/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor   

...................................................................................................
llama_new_context_with_model: n_ctx      = 4000
llama_new_context_with_model: freq_base  = 10000.0
llama_new_context_with_model: freq_scale = 1
llama_kv_cache_init: offloading v cache to GPU
llama_kv_cache_init: offloading k cache to GPU
llama_kv_cache_init: VRAM kv self = 2000.00 MB
llama_new_context_with_model: kv self size  = 2000.00 MB
llama_build_graph: non-view tensors processed: 740/740
llama_new_context_with_model: compute buffer total size = 288.44 MB
llama_new_context_with_model: VRAM scratch buffer: 281.82 MB
llama_new_context_with_model: total VRAM used: 6756.75 MB (model: 4474.93 MB, context: 2281.82 MB)


# Prompting Pipeline
The following are all the functions required to communicate with the LLM to fulfill the prompt/thought/response framework. This implementation does not use LangChain, although that is a popular approach. Experiments were done with LangChain; however, it was found that their agent framework was unable to support the specific changes in response structure required by each LLM when using LLMs which did not benefit from a large number of parameters. Therefore, a custom implementation was built.

This example represents a chat bot which effectively responds to friendly conversation, filters out unreleated questions to Next Step Clinic's mission, and builds a user profile throughout the conversation.

In [11]:
def intake_user_prompt(user_prompt):
    """
    Given a string from a user, put into the JSON format the LLM expects
    :param str user_prompt: Direct input from user
    :return: JSON format
    """
    user_response = {
        'role':'user',
        'content':user_prompt
    }
    return user_response

In [12]:
def finalize_message_content(message):
    """
    Final filtration of unprofessional language from the chat-bot
    :param str message: What the chat-bot would have responded with
    :return: What will actually be output
    """
    emojis = "😊😀😃😄😁🥦🥕🌽😆😅😂🤣😊🥨🍞😇😉🌱😌😍🌳🥨🥝🥗🥝🍵🥜🍔🌈🥘🥚😘🥝💪😗😙😚🤗💪🤔😐😑😶🙄😏😣😥😮😯😪😫😴😌😛😜😝🤤😒😓😔😕🙃🤑😲☹️🙁😖😞😟😤😢😭😦😧😨😩😬😰😱😳😵😡😠😷🤒🤕🤢🤮🤧😇🤠🤡🤥🤫🤭🧐🤓😈👿👹👺💀☠️👻👽👾🤖🎃😺😸😹😻😼😽🙀😿😾🤲🤞🤟🤘🤙👌👍👎✊✌️🤛🤜👊🤝👏🙌👐🤲🤝🤞🤟🤠👑🤰🤱👶🧒👦👧👨👩🧑👱‍♂️👱‍♀️👴👵🙍‍♂️🙍‍♀️🙎‍♂️🙎‍♀️🙅‍♂️🙅‍♀️🙆‍♂️🙆‍♀️💁‍♂️💁‍♀️🙋‍♂️🙋‍😊🌟🤓🎨🎭📚📖🤝😅💪🤔🌟💕👥💬📢💡🎯🔍🏼🌈🎉💭📝💕🎬💻💖🤖✈🚀"
    resulting_string = ''.join(char for char in message if char not in emojis)
    pattern = r'\*([^*]+)\*' # Remove *__*; action terms
    resulting_string2 = re.sub(pattern, '' , resulting_string)
    return resulting_string2

In [13]:
def generate_llm_response(llm, chat_history, temperature=TEMPERATURE):
    """
    Provided an LLM and its chat history (including the most recent user prompt),
    retrieve the model's inference for response.
    :param llm: LLM to generate/inference responses from
    :param dict chat_history: JSON format of {'role':'user/assistant/system', 'content':'...'}
    :param float temperature: Temperature to set LLM response (default = TEMPERATURE constant)
    :return: dict representing LLM's response message
    """
    messages = [{'role': str(item['role']), 'content': str(item['content'])} for item in chat_history]
    resp_msg = {'role': '', 'content': ''} 
    while resp_msg['content'] == '': # Repeat until not a blank response
        resp_stream = llm.create_chat_completion(messages, stream=True, temperature=temperature)
        for tok in resp_stream:
            delta = tok['choices'][0]['delta']
            for key, value in delta.items():
                if isinstance(value, str):
                    resp_msg[key] += value
                else: 
                    resp_msg[key] += str(value)
    return resp_msg

In [14]:
def ask_filtration_llm(llm, user_prompt):
    """
    Determine if the user's request should be filtered before reaching the chat-bot
    :param llm: LLM responsible for responding
    :param str user_prompt: Message from the user
    :return: True if should be filtered, False if not
    """
    # Set up the chat history with the response filter LLM config
    filter_history = []
    filter_history.append(RESPONSE_FILTER_LLM_SYSTEM_CONFIG)
    
    # Change the text is a way where it's easier for the filter to understand
    prompt_to_filter = "Should the following user question be filtered?: " + str(user_prompt.get('content', ''))
    prompt_to_filter = {'role':'user', 'content':prompt_to_filter}
    filter_history.append(prompt_to_filter)

    resp_msg = generate_llm_response(llm, filter_history)
    print("FILTER LLM RESPONSE: ", resp_msg)
    print("\n")
    
    # Extract the content from the response message
    content = resp_msg['content']

    # Parse the response of the LLM for yes/no
    pattern_yes = re.compile(r'\byes\b', re.IGNORECASE)
    pattern_no = re.compile(r'\bno\b', re.IGNORECASE)
    match_yes = pattern_yes.search(content)
    match_no = pattern_no.search(content)

    # Check which occurs first
    if match_yes and match_no:
        if match_yes.start() < match_no.start():
            return True  # "'yes' is the first to occur"
        else:
            return False  # "'no' is the first to occur"
    elif match_yes:
        return True  # "'yes' is the first to occur"
    elif match_no:
        return False  # "'no' is the first to occur"
    else:
        return False  # "Neither 'yes' nor 'no' is in the string"

In [15]:
def build_user_profile(llm, chat_history):
    """
    Based on chat_history, build a description of the user following the guidelines 
    outlined in the system config PROFILE_BUILDER_LLM_SYSTEM_CONFIG
    :param llm: LLM responsible for responding
    :param dict chat_history: JSON format of {'role':'user/assistant/system', 'content':'...'}
    :return: string representing user's profile
    """
    profile_build_history = []
    profile_build_history.append(PROFILE_BUILDER_LLM_SYSTEM_CONFIG)
    profile_build_history.append({'role':'user', 'content':"Build a user profile from the following conversation: "+str(chat_history)})
    
    resp_msg = generate_llm_response(llm, profile_build_history)
    print("USER PROFILE LLM RESPONSE", resp_msg['content'])
    print("\n")
    return resp_msg['content']

In [16]:
def send_message_from_user(llm, user_message, chat_history):
    """
    Send a message from the user to the LLM speaker/agent framework
    :param llm: LLM for all tasks, including "thought" processes
    :param str user_message: Message from the user
    :param dict chat_history: Previous messages in conversation and system prompt
    :return: Newest message from the chat-bot
    """
    # "USER PROMPT"
    print("==================================")
    user_prompt = intake_user_prompt(user_message)
    resp_msg = {'role': '', 'content': ''} 

    # "THOUGHTS"
    if ask_filtration_llm(llm, user_prompt):
        SET_FILTER_RESPONSE = """I'm sorry, but I'm a chat-bot dedicated to answering questions 
        regarding commodity food price warnings and the services that the United Nation's Food and Agriculture Organization (FAO) provides. 
        I'm unable to answer your question at this time."""
        chat_history.append({'role':'assistant', 'content':SET_FILTER_RESPONSE})
    else:
        chat_history.append(user_prompt) # add user input to history
        chat_history.append(generate_llm_response(llm, chat_history))
        
    print("=|Chat-Bot Message|===============================")
    print("FINAL CHATBOT MESSAGE: ", finalize_message_content(chat_history[-1]['content']))
    print("==================================" + '\n')
    return chat_history

In [17]:
def start_conversation(chat_history, llm):
    """
    Begin the repeating conversation between the user and the communicative LLM
    :param dict chat_history: Conversation that is building with LLM
    :param llama-2-7b llm: LLM being used for speaker/agend LLM framework
    """
    i = 0
    
    while i >= 0:
        if i == 0:
            chat_history = send_message_from_user(llm, COMMUNICATIVE_LLM_SYSTEM_CONFIG, chat_history)
        print("=|User Message|==============================")
        chat_history = send_message_from_user(llm, input(), chat_history)
        print("\n")
        i+=1

In [ ]:
# Chat history is the interface that allows us to track the conversation as the user and chat-bot interact.
# By using this JSON framework, we are able to recognize the conversations in previous statements.
chat_history = []
# chat_history.append(COMMUNICATIVE_LLM_SYSTEM_CONFIG) # Add the system prompt so the LLM is aware of how it is supposed to "act"
start_conversation(chat_history, LLM)

FILTER LLM RESPONSE:  {'role': 'assistantassistant', 'content': "  No, the user question should not be filtered. The user is asking for advice on food security and price warnings in their country, which is relevant and appropriate for the chat bot's purpose. The user is also from Argentina, which is currently under a high warning for corn, making their question even more relevant. The chat bot should respond with concise and targeted advice to help the user understand the situation and take appropriate action."}


=|Chat-Bot Message|===============================
FINAL CHATBOT MESSAGE:    Hello! I'm here to help you with any questions or concerns you may have regarding food security in Argentina and around the world. As a representative of the United Nations Food and Agriculture Organization (FAO), I've been monitoring the Global Information and Early Warning System on Food and Agriculture (GIEWS) and I wanted to reach out to you directly because your country is currently under a high